In [1]:
import pandas as pd 
import json 
import requests
import os 
from dotenv import load_dotenv
import sys 
from itertools import permutations
import folium


def print_json(data:json):
    """json 형태의 데이터를 출력하는 함수입니다.

    Args:
        data (json): json 형태의 데이터
    """
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

def find_target_directory(target_dir_name):
    current_path = os.getcwd()
    while True:
        # 상위 디렉토리에서 대상 디렉토리 이름을 찾음
        parent_path, current_dir = os.path.split(current_path)
        
        if current_dir == target_dir_name:
            return current_path
        
        if parent_path == current_path:  # 최상위 디렉토리에 도달하면 종료
            break
        
        # 상위 디렉토리로 이동
        current_path = parent_path
    
    return None  # 대상 디렉토리를 찾지 못한 경우


def get_project_root_path(proejct_directory_name: str='odysseyes'):
    return  find_target_directory(target_dir_name=proejct_directory_name)


project_root_path = get_project_root_path()
print(f"프로젝트 경로: {project_root_path}")
sys.path.append(os.path.join(project_root_path, 'recommend', 'func'))

# from tmap_route_optimizer import TMAPClient, RouteOptimizer, PlaceDataManager
from tmap_client import TMAPClient

프로젝트 경로: c:\Users\SSAFY\Desktop\workspace\odysseyes


In [2]:
load_dotenv()
api_key = os.getenv('SK_OPEN_API_KEY')
tmap_client = TMAPClient(api_key=api_key)

In [3]:
# 장소 데이터를 예시로 입력 (출발지, 경유지 등)

# ['대전 서구 월평동 1216', '무드빌리지', '장원막국수', '백제문화단지']

places = [
    {"name": "대전 서구 월평동 1216", "lat": None, "lon": None},
    {"name": "부여 무드빌리지", "lat": None, "lon": None},
    {"name": "부여 장원막국수", "lat": None, "lon": None},
    {"name": "부여 백제문화단지", "lat": None, "lon": None},
    # ... 추가 경유지
]

In [4]:
# 거리 및 경로 데이터를 가져오는 함수 (예시용 API 엔드포인트 사용)
def fetch_route_data(tmap_client:TMAPClient, origin, destination):
    origin_poi = tmap_client.get_poi(origin['name'])
    destination_poi = tmap_client.get_poi(destination['name'])

    # url = "https://api.example.com/get_route"
    # params = {
    #     "origin": f"{origin_poi['latitude']},{origin_poi['longitude']}",
    #     "destination": f"{destination_poi['latitude']},{destination_poi['longitude']}"
    # }
    # response = requests.get(url, params=params)

    response = tmap_client.get_route_data(start=origin_poi, end=destination_poi)
    return response  # {"distance": km, "time": min, "cost": won, "polyline": ...}


In [5]:
# 모든 장소 간 1:1 거리, 시간, 비용 데이터를 수집
routes_data = {}
for i, origin in enumerate(places):
    # print(origin)
    for j, destination in enumerate(places):
        # print(destination)
        # break
        if i != j:
            routes_data[(i, j)] = fetch_route_data(tmap_client=tmap_client, origin=origin, destination=destination)

In [6]:
print(len(routes_data))

12


In [56]:
from sklearn.preprocessing import MinMaxScaler

def get_scaled_properties(routes:dict):
    # routes = list(routes.values())
    route_dict = []
    for k, v in routes.items():    
        print(type(v))
        print(routes)
        # print(type(routes[0])) 
        route_dict.append(v)
 
    all_features_list = [route['features'][0] for route in route_dict]  # 모든 1대1 경로들의 features 리스트
    all_properties_list = [features['properties'] for features in all_features_list]  # 모든 1대1 경로들의 경로 요약 정보 

    # 경로들의 전체 거리, 소요 시간, 비용만 추출한 리스트 
    selected_properties_data = [
        {
            'totalDistance': properties['totalDistance'],
            'totalTime': properties['totalTime'],
            'totalFare': properties['totalFare'] 
        }
        for properties in all_properties_list
    ]

    scaler = MinMaxScaler()

    scaled_selected_properties_data = scaler.fit_transform(pd.DataFrame(selected_properties_data))
    scaled_scores = []
    for i in range(len(scaled_selected_properties_data[0])):
        scaled_score = []
        for j in range(len(scaled_selected_properties_data)):
            scaled_score.append(round(float(scaled_selected_properties_data[j][i]), 3))
        scaled_scores.append(scaled_score)
    
    scaledProperties = []
    totalRouteScores = []
    for i in range(len(scaled_scores[0])):
        scaledProperty = {
            'scaledDistance': round(1 - scaled_scores[0][i], 2),
            'scaledTime': round(1 - scaled_scores[1][i], 2),
            'scaledFare': round(1 - scaled_scores[2][i], 2),
            # 'scaledPlaceScore': scaled_scores[3][i],
        }
        
        scaledDTFScore = round(sum(scaledProperty.values()), 2)
        scaledProperty['scaledDTFScore'] = scaledDTFScore
        # totalRouteScores.append(totalRouteScore)
        scaledProperties.append(scaledProperty)

    for i, route in enumerate(routes):
        routes[i]['features'][0]['properties']['scaledProperties'] = scaledProperties[i]
    
    print_json(routes)
    return routes


# 각 경로에 대한 점수를 계산하는 함수
def calculate_route_score(route, routes_data, weight_distance=0.4, weight_time=0.4, weight_fare=0.2):
    print(route)
    score = 0
    for i in range(len(route) - 1):
        route_info = routes_data[(route[i], route[i + 1])]
        # print(0000)
        # print(type(route_info))
        route_info = get_scaled_properties(route_info)
        route_features = route_info['features'][0]
        # print_json(route_info)
        route_properties = route_features['properties']
        route_info_scaled_properties = route_properties['scaledProperties']
        # print(route_info_scaled_properties)
        score += (
            weight_distance * route_info_scaled_properties["scaledDistance"] +
            weight_time * route_info_scaled_properties["scaledTime"] +
            weight_fare * route_info_scaled_properties["scaledFare"]
        )
    return score




# 출발지에서 시작하여 모든 장소를 방문 후 출발지로 돌아오는 최적 경로 탐색
def find_optimal_route(places, routes_data):
    num_places = len(places)
    all_routes = permutations(range(1, num_places))  # 출발지를 제외한 경유지들의 순열 생성
    best_score = 0
    best_route = None

    print(f'출발지 제외 경유지 순열: {list(all_routes)}')   

    for route in all_routes:
        route = (0,) + route + (0,)  # 출발지(0)를 추가해 순환 경로 형성
        # print(route)
        score = calculate_route_score(route, routes_data)
        # print(f'{route}: {score}')
        if score > best_score:
            best_score = score
            best_route = route
            
    return best_route, best_score

In [57]:
test_route = (0,1,2,3,0)
for i in range(5-1):
    print(f"{ test_route[i] } -> { test_route[i + 1] }")
    route_info = get_scaled_properties(routes_data[(test_route[i], test_route[i + 1])])
    # print_json(route_info)
    # print(len(route_info))
    features = route_info['features'][0]
    # print_json(features)
    properties = features['properties']
    print_json(properties)
    # print_json(route_info['properties']['scaledProperties'])   

0 -> 1
<class 'str'>
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [127.35686486460436, 36.35506787940311]}, 'properties': {'totalDistance': 59374, 'totalTime': 4107, 'totalFare': 0, 'taxiFare': 58020, 'index': 0, 'pointIndex': 0, 'name': '', 'description': '월평새뜸로4번길 을 따라  방면으로 240m 이동', 'nextRoadName': '월평새뜸로4번길', 'turnType': 200, 'pointType': 'S'}}, {'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[127.35686486460436, 36.35506787940311], [127.35710929478405, 36.35478736029347], [127.35751760423508, 36.35432075428017], [127.35761482144842, 36.35418466047175], [127.35765926680916, 36.354004126370235], [127.35773982388905, 36.35368194245708], [127.35771482710366, 36.35364583503596], [127.35723431716806, 36.35354861542231], [127.35707322182306, 36.35349861828908]]}, 'properties': {'index': 1, 'lineIndex': 0, 'name': '월평새뜸로4번길', 'description': '월평새뜸로4번길, 240m', 'distance': 240, 'time': 14, 'roadType': 9, '

TypeError: string indices must be integers

In [30]:
# 최적 경로 탐색 수행
best_route, best_score = find_optimal_route(places, routes_data)
print("최적 경로:", best_route)
print("최적 점수:", best_score)

출발지 제외 경유지 순열: [(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]
최적 경로: None
최적 점수: 0


In [ ]:

# 지도 시각화 예시
m = folium.Map(location=[places[0]["lat"], places[0]["lon"]], zoom_start=13)

# 최적 경로 표시
for i in range(len(best_route) - 1):
    start = places[best_route[i]]
    end = places[best_route[i + 1]]
    polyline = routes_data[(best_route[i], best_route[i + 1])]["polyline"]
    folium.PolyLine(locations=polyline, color="blue", weight=2.5, opacity=1).add_to(m)

# 시작 및 끝 지점 마커 추가
folium.Marker([places[0]["lat"], places[0]["lon"]], popup="출발지").add_to(m)

# 지도 출력
m
